<a href="https://colab.research.google.com/github/Edakksm/Deep-Learning/blob/master/BERT_Smitha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pytorch-pretrained-bert pytorch-nlp

import tensorflow as tf
import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertForSequenceClassification, BertAdam, BertConfig, BertTokenizer
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 133kB 4.8MB/s 
     |████████████████████████████████| 92kB 11.5MB/s 


Using TensorFlow backend.


In [6]:
device_name = tf.test.gpu_device_name
print(device_name)
if device_name != '/device:GPU:0':
  print("GPU Not found")
else:
  print("GPU Found")

<function gpu_device_name at 0x7f0b9f882ea0>
GPU Not found


In [0]:
df = pd.read_csv('in_domain_train.tsv', delimiter='\t', header=None, names=['sentence_source','label','label_notes','sentence'])

In [8]:
df.head()

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.


In [9]:
df.shape

(8551, 4)

In [10]:
df.sample(10)

,sentence_source,label,label_notes,sentence
8486,ad03,1,NaN,Nathan caused Benjamin to see himself in the m...
2205,l-93,1,NaN,Tessa cut herself.
8131,ad03,0,*,She kicked itself
1642,r-67,0,*,I brought a razor to shave myself with.
8228,ad03,0,*,The scissors is lost
5761,c_13,1,NaN,I didn't read a single book the whole time I w...
5371,b_73,0,*,Sally eats more the stuff.
1004,bc01,1,NaN,I showed every boy a planet.
4233,ks08,1,NaN,All of his wealth come from real estate invest...
8045,ad03,0,*,letter is on the table


In [0]:
sentences = df.sentence

In [12]:
sentences

0       Our friends won't buy this analysis, let alone...
1       One more pseudo generalization and I'm giving up.
2        One more pseudo generalization or I'm giving up.
3          The more we study verbs, the crazier they get.
4               Day by day the facts are getting murkier.
                              ...                        
8546                     Poseidon appears to own a dragon
8547                       Digitize is my happiest memory
8548                       It is easy to slay the Gorgon.
8549         I had the strangest feeling that I knew you.
8550                  What all did you get for Christmas?
Name: sentence, Length: 8551, dtype: object

In [0]:
sentences = [ "[CLS] " + sentence + " [SEP]" for sentence in sentences]

In [14]:
sentences[0]

"[CLS] Our friends won't buy this analysis, let alone the next one we propose. [SEP]"

In [0]:
# Import BERT Tokenizer

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
tokenized_texts[0]

100%|██████████| 231508/231508 [00:00<00:00, 1145856.61B/s]


['[CLS]',
 'our',
 'friends',
 'won',
 "'",
 't',
 'buy',
 'this',
 'analysis',
 ',',
 'let',
 'alone',
 'the',
 'next',
 'one',
 'we',
 'propose',
 '.',
 '[SEP]']

In [17]:
print(tokenized_texts[0])

['[CLS]', 'our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.', '[SEP]']


In [0]:
# Pad sequences

In [0]:
MAX_LEN = 128

In [0]:
input_ids = [ tokenizer.convert_tokens_to_ids(token) for token in tokenized_texts]

In [21]:
print(input_ids[0])

[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]


In [22]:
len(input_ids[0])

19

In [23]:
len(input_ids[1])

14

In [0]:
input_ids = pad_sequences(input_ids, MAX_LEN, padding="post", truncating="post", dtype="long")

In [25]:
len(input_ids[0])

128

In [26]:
print(input_ids[0])

[  101  2256  2814  2180  1005  1056  4965  2023  4106  1010  2292  2894
  1996  2279  2028  2057 16599  1012   102     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [0]:
attention_masks =[]
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [28]:
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [0]:
# split into train and validation sets
labels = df.label.values
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)

In [30]:
train_inputs.shape

(7695, 128)

In [0]:
# Convert all data into torch sensors

In [32]:
train_inputs[0:2]

array([[  101,  2002,  2939,  1996,  3328,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [36]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [0]:
train_masks, validation_masks,_,_ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.2)

In [0]:
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [45]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequntialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler= validation_sampler, batch_size = batch_size)

AssertionError: ignored

In [0]:
### Train model

In [47]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

100%|██████████| 407873900/407873900 [00:11<00:00, 34071865.30B/s]


In [48]:
model.cuda

<bound method Module.cuda of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Drop

In [0]:
param_optimizer = list(model.named_parameters())

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [52]:
optmizer = BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)

t_total value of -1 results in schedule not being applied


In [0]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)